In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from catboost import Pool, CatBoostRegressor, cv
from sklearn.metrics import mean_squared_error
%matplotlib inline


In [47]:
def make_submission(test, y_p, path):
    pd.DataFrame(list(zip(test.index, y_p))).to_csv(path, index=False, header=['unique_id', 'target'])

In [39]:
categorical_features = [
    'Организатор',
    'Дата начала подачи предложений',
    'Тип методу закупівлі',
    'Тип организатора',
     'Статус лота',
     'СтатусТендера', 
     'РегионОрганизатора',
     'Регион поставки',
     'Тендер',
     'Дата начала периода уточнений',
     'Дата завершения',
     'Дата завершения подачи предложений']

def getCatFeaturesIds(features):
    ids = []
    for i, f in enumerate(features):
        if f in categorical_features:
            ids.append(i)
        
    return ids
    

In [35]:
def fillMissing(data):
    data.loc[data['СуммаГарантииПредложенияЛота'] == '-', "СуммаГарантииПредложенияЛота"] = 0

def preprocess_train(train_data):
    train_clean = train_data.drop(['IDOrganizator', 'РегионУчастника', 'ID'], axis = 1)
    return train_clean

def preprocess_test(test_data):
    return test_data.drop(['IDOrganizator'], axis = 1)    

def extract_target(train_data):
    return train_data['ID']
    

In [36]:
# Read data
train_data = pd.read_csv('../data/train_agg_2.csv', index_col = 0)
train_labels = extract_target(train_data)
test_data = pd.read_csv('../data/test.csv', index_col = 0)
fillMissing(train_data)
fillMissing(test_data)

In [40]:
train_p = preprocess_train(train_data)
test_p = preprocess_test(test_data)
cats = list(range(2, 14))

In [41]:
cats = getCatFeaturesIds(list(train_p.columns))

In [58]:
train_pool = Pool(train_p, train_labels, cat_features=cats)
test_pool = Pool(test_p, cat_features=cats)

In [77]:
model = CatBoostRegressor(iterations=6000, depth=5,
                          l2_leaf_reg=0.1, leaf_estimation_method='Gradient',
                          bagging_temperature=1, 
                          learning_rate=0.2, loss_function="RMSE")

In [78]:
model.fit(train_pool)

0:	learn: 2.0040181	total: 371ms	remaining: 37m 5s
1:	learn: 1.7637537	total: 648ms	remaining: 32m 24s
2:	learn: 1.5902766	total: 861ms	remaining: 28m 40s
3:	learn: 1.4683825	total: 1.03s	remaining: 25m 41s
4:	learn: 1.3772191	total: 1.28s	remaining: 25m 36s
5:	learn: 1.3200161	total: 1.53s	remaining: 25m 33s
6:	learn: 1.2808520	total: 1.71s	remaining: 24m 21s
7:	learn: 1.2525230	total: 1.91s	remaining: 23m 53s
8:	learn: 1.2283653	total: 2.36s	remaining: 26m 12s
9:	learn: 1.2116662	total: 2.79s	remaining: 27m 53s
10:	learn: 1.2015789	total: 3.08s	remaining: 27m 55s
11:	learn: 1.1952785	total: 3.33s	remaining: 27m 40s
12:	learn: 1.1913420	total: 3.53s	remaining: 27m 6s
13:	learn: 1.1864254	total: 3.9s	remaining: 27m 48s
14:	learn: 1.1811828	total: 4.15s	remaining: 27m 36s
15:	learn: 1.1772699	total: 4.32s	remaining: 26m 56s
16:	learn: 1.1746205	total: 4.46s	remaining: 26m 11s
17:	learn: 1.1713999	total: 4.76s	remaining: 26m 21s
18:	learn: 1.1699594	total: 4.9s	remaining: 25m 43s
19:	lea

156:	learn: 1.1102615	total: 47.8s	remaining: 29m 37s
157:	learn: 1.1100820	total: 48.1s	remaining: 29m 38s
158:	learn: 1.1099624	total: 48.5s	remaining: 29m 39s
159:	learn: 1.1098542	total: 48.8s	remaining: 29m 40s
160:	learn: 1.1097812	total: 49.2s	remaining: 29m 42s
161:	learn: 1.1096245	total: 49.6s	remaining: 29m 47s
162:	learn: 1.1094489	total: 49.8s	remaining: 29m 44s
163:	learn: 1.1093670	total: 50.2s	remaining: 29m 44s
164:	learn: 1.1093609	total: 50.4s	remaining: 29m 43s
165:	learn: 1.1092645	total: 50.7s	remaining: 29m 42s
166:	learn: 1.1091618	total: 51s	remaining: 29m 41s
167:	learn: 1.1091134	total: 51.3s	remaining: 29m 42s
168:	learn: 1.1090387	total: 51.7s	remaining: 29m 44s
169:	learn: 1.1089526	total: 52s	remaining: 29m 44s
170:	learn: 1.1088837	total: 52.4s	remaining: 29m 44s
171:	learn: 1.1087947	total: 52.7s	remaining: 29m 45s
172:	learn: 1.1087281	total: 53s	remaining: 29m 46s
173:	learn: 1.1086077	total: 53.3s	remaining: 29m 45s
174:	learn: 1.1085541	total: 53.7s

307:	learn: 1.0990349	total: 1m 36s	remaining: 29m 40s
308:	learn: 1.0989757	total: 1m 36s	remaining: 29m 39s
309:	learn: 1.0988765	total: 1m 36s	remaining: 29m 37s
310:	learn: 1.0987903	total: 1m 37s	remaining: 29m 37s
311:	learn: 1.0987444	total: 1m 37s	remaining: 29m 36s
312:	learn: 1.0987224	total: 1m 37s	remaining: 29m 36s
313:	learn: 1.0986331	total: 1m 38s	remaining: 29m 36s
314:	learn: 1.0986111	total: 1m 38s	remaining: 29m 36s
315:	learn: 1.0984916	total: 1m 38s	remaining: 29m 35s
316:	learn: 1.0983843	total: 1m 39s	remaining: 29m 35s
317:	learn: 1.0983740	total: 1m 39s	remaining: 29m 34s
318:	learn: 1.0983520	total: 1m 39s	remaining: 29m 32s
319:	learn: 1.0983126	total: 1m 39s	remaining: 29m 31s
320:	learn: 1.0982554	total: 1m 40s	remaining: 29m 30s
321:	learn: 1.0982275	total: 1m 40s	remaining: 29m 31s
322:	learn: 1.0981460	total: 1m 40s	remaining: 29m 30s
323:	learn: 1.0981166	total: 1m 41s	remaining: 29m 30s
324:	learn: 1.0979514	total: 1m 41s	remaining: 29m 32s
325:	learn

458:	learn: 1.0922596	total: 2m 23s	remaining: 28m 51s
459:	learn: 1.0922135	total: 2m 23s	remaining: 28m 51s
460:	learn: 1.0921540	total: 2m 24s	remaining: 28m 51s
461:	learn: 1.0921094	total: 2m 24s	remaining: 28m 51s
462:	learn: 1.0920569	total: 2m 24s	remaining: 28m 51s
463:	learn: 1.0920043	total: 2m 25s	remaining: 28m 51s
464:	learn: 1.0919825	total: 2m 25s	remaining: 28m 52s
465:	learn: 1.0919611	total: 2m 25s	remaining: 28m 52s
466:	learn: 1.0919485	total: 2m 26s	remaining: 28m 52s
467:	learn: 1.0919435	total: 2m 26s	remaining: 28m 52s
468:	learn: 1.0919172	total: 2m 26s	remaining: 28m 53s
469:	learn: 1.0919050	total: 2m 27s	remaining: 28m 52s
470:	learn: 1.0918728	total: 2m 27s	remaining: 28m 51s
471:	learn: 1.0918625	total: 2m 27s	remaining: 28m 50s
472:	learn: 1.0918411	total: 2m 28s	remaining: 28m 51s
473:	learn: 1.0918208	total: 2m 28s	remaining: 28m 51s
474:	learn: 1.0918145	total: 2m 28s	remaining: 28m 50s
475:	learn: 1.0917949	total: 2m 29s	remaining: 28m 49s
476:	learn

608:	learn: 1.0867566	total: 3m 12s	remaining: 28m 20s
609:	learn: 1.0867409	total: 3m 12s	remaining: 28m 19s
610:	learn: 1.0866799	total: 3m 12s	remaining: 28m 19s
611:	learn: 1.0866710	total: 3m 12s	remaining: 28m 19s
612:	learn: 1.0866503	total: 3m 13s	remaining: 28m 19s
613:	learn: 1.0866454	total: 3m 13s	remaining: 28m 19s
614:	learn: 1.0866211	total: 3m 13s	remaining: 28m 18s
615:	learn: 1.0865588	total: 3m 14s	remaining: 28m 18s
616:	learn: 1.0865409	total: 3m 14s	remaining: 28m 18s
617:	learn: 1.0865313	total: 3m 15s	remaining: 28m 19s
618:	learn: 1.0865136	total: 3m 15s	remaining: 28m 18s
619:	learn: 1.0865031	total: 3m 15s	remaining: 28m 18s
620:	learn: 1.0864747	total: 3m 16s	remaining: 28m 18s
621:	learn: 1.0864462	total: 3m 16s	remaining: 28m 18s
622:	learn: 1.0864271	total: 3m 16s	remaining: 28m 18s
623:	learn: 1.0863913	total: 3m 17s	remaining: 28m 19s
624:	learn: 1.0863796	total: 3m 17s	remaining: 28m 18s
625:	learn: 1.0863676	total: 3m 17s	remaining: 28m 19s
626:	learn

...

5310:	learn: 1.0212104	total: 29m 10s	remaining: 3m 47s
5311:	learn: 1.0212047	total: 29m 10s	remaining: 3m 46s
5312:	learn: 1.0212015	total: 29m 10s	remaining: 3m 46s
5313:	learn: 1.0211977	total: 29m 11s	remaining: 3m 46s
5314:	learn: 1.0211901	total: 29m 11s	remaining: 3m 45s
5315:	learn: 1.0211754	total: 29m 11s	remaining: 3m 45s
5316:	learn: 1.0211711	total: 29m 12s	remaining: 3m 45s
5317:	learn: 1.0211560	total: 29m 12s	remaining: 3m 44s
5318:	learn: 1.0211405	total: 29m 12s	remaining: 3m 44s
5319:	learn: 1.0211239	total: 29m 13s	remaining: 3m 44s
5320:	learn: 1.0211106	total: 29m 13s	remaining: 3m 43s
5321:	learn: 1.0210782	total: 29m 13s	remaining: 3m 43s
5322:	learn: 1.0210601	total: 29m 14s	remaining: 3m 43s
5323:	learn: 1.0210416	total: 29m 14s	remaining: 3m 42s
5324:	learn: 1.0209995	total: 29m 14s	remaining: 3m 42s
5325:	learn: 1.0209952	total: 29m 15s	remaining: 3m 42s
5326:	learn: 1.0209860	total: 29m 15s	remaining: 3m 41s
5327:	learn: 1.0209820	total: 29m 15s	remaining:

5458:	learn: 1.0197773	total: 29m 59s	remaining: 2m 58s
5459:	learn: 1.0197681	total: 30m	remaining: 2m 58s
5460:	learn: 1.0197486	total: 30m	remaining: 2m 57s
5461:	learn: 1.0197395	total: 30m	remaining: 2m 57s
5462:	learn: 1.0197296	total: 30m 1s	remaining: 2m 57s
5463:	learn: 1.0197205	total: 30m 1s	remaining: 2m 56s
5464:	learn: 1.0197069	total: 30m 1s	remaining: 2m 56s
5465:	learn: 1.0196957	total: 30m 2s	remaining: 2m 56s
5466:	learn: 1.0196887	total: 30m 2s	remaining: 2m 55s
5467:	learn: 1.0196770	total: 30m 2s	remaining: 2m 55s
5468:	learn: 1.0196599	total: 30m 3s	remaining: 2m 55s
5469:	learn: 1.0196490	total: 30m 3s	remaining: 2m 54s
5470:	learn: 1.0196430	total: 30m 3s	remaining: 2m 54s
5471:	learn: 1.0196381	total: 30m 4s	remaining: 2m 54s
5472:	learn: 1.0196313	total: 30m 4s	remaining: 2m 53s
5473:	learn: 1.0196249	total: 30m 4s	remaining: 2m 53s
5474:	learn: 1.0196210	total: 30m 5s	remaining: 2m 53s
5475:	learn: 1.0196050	total: 30m 5s	remaining: 2m 52s
5476:	learn: 1.019

5605:	learn: 1.0183981	total: 30m 48s	remaining: 2m 9s
5606:	learn: 1.0183908	total: 30m 49s	remaining: 2m 9s
5607:	learn: 1.0183778	total: 30m 49s	remaining: 2m 9s
5608:	learn: 1.0183709	total: 30m 49s	remaining: 2m 8s
5609:	learn: 1.0183531	total: 30m 49s	remaining: 2m 8s
5610:	learn: 1.0183470	total: 30m 50s	remaining: 2m 8s
5611:	learn: 1.0183330	total: 30m 50s	remaining: 2m 7s
5612:	learn: 1.0183239	total: 30m 51s	remaining: 2m 7s
5613:	learn: 1.0183132	total: 30m 51s	remaining: 2m 7s
5614:	learn: 1.0183053	total: 30m 51s	remaining: 2m 6s
5615:	learn: 1.0183027	total: 30m 51s	remaining: 2m 6s
5616:	learn: 1.0182913	total: 30m 52s	remaining: 2m 6s
5617:	learn: 1.0182790	total: 30m 52s	remaining: 2m 5s
5618:	learn: 1.0182712	total: 30m 52s	remaining: 2m 5s
5619:	learn: 1.0182624	total: 30m 53s	remaining: 2m 5s
5620:	learn: 1.0182567	total: 30m 53s	remaining: 2m 4s
5621:	learn: 1.0182513	total: 30m 53s	remaining: 2m 4s
5622:	learn: 1.0182411	total: 30m 54s	remaining: 2m 4s
5623:	lear

5753:	learn: 1.0169803	total: 31m 38s	remaining: 1m 21s
5754:	learn: 1.0169744	total: 31m 39s	remaining: 1m 20s
5755:	learn: 1.0169718	total: 31m 39s	remaining: 1m 20s
5756:	learn: 1.0169651	total: 31m 39s	remaining: 1m 20s
5757:	learn: 1.0169540	total: 31m 40s	remaining: 1m 19s
5758:	learn: 1.0169486	total: 31m 40s	remaining: 1m 19s
5759:	learn: 1.0169434	total: 31m 40s	remaining: 1m 19s
5760:	learn: 1.0169358	total: 31m 41s	remaining: 1m 18s
5761:	learn: 1.0169259	total: 31m 41s	remaining: 1m 18s
5762:	learn: 1.0169151	total: 31m 41s	remaining: 1m 18s
5763:	learn: 1.0169118	total: 31m 42s	remaining: 1m 17s
5764:	learn: 1.0169046	total: 31m 42s	remaining: 1m 17s
5765:	learn: 1.0168949	total: 31m 42s	remaining: 1m 17s
5766:	learn: 1.0168893	total: 31m 43s	remaining: 1m 16s
5767:	learn: 1.0168841	total: 31m 43s	remaining: 1m 16s
5768:	learn: 1.0168755	total: 31m 43s	remaining: 1m 16s
5769:	learn: 1.0168694	total: 31m 44s	remaining: 1m 15s
5770:	learn: 1.0168656	total: 31m 44s	remaining:

5903:	learn: 1.0155620	total: 32m 28s	remaining: 31.7s
5904:	learn: 1.0155542	total: 32m 29s	remaining: 31.4s
5905:	learn: 1.0155385	total: 32m 29s	remaining: 31s
5906:	learn: 1.0155303	total: 32m 29s	remaining: 30.7s
5907:	learn: 1.0155164	total: 32m 30s	remaining: 30.4s
5908:	learn: 1.0155120	total: 32m 30s	remaining: 30s
5909:	learn: 1.0155028	total: 32m 30s	remaining: 29.7s
5910:	learn: 1.0154977	total: 32m 31s	remaining: 29.4s
5911:	learn: 1.0154919	total: 32m 31s	remaining: 29s
5912:	learn: 1.0154847	total: 32m 31s	remaining: 28.7s
5913:	learn: 1.0154677	total: 32m 32s	remaining: 28.4s
5914:	learn: 1.0154597	total: 32m 32s	remaining: 28.1s
5915:	learn: 1.0154557	total: 32m 32s	remaining: 27.7s
5916:	learn: 1.0154489	total: 32m 33s	remaining: 27.4s
5917:	learn: 1.0154263	total: 32m 33s	remaining: 27.1s
5918:	learn: 1.0154175	total: 32m 33s	remaining: 26.7s
5919:	learn: 1.0154129	total: 32m 33s	remaining: 26.4s
5920:	learn: 1.0154112	total: 32m 34s	remaining: 26.1s
5921:	learn: 1.0

In [79]:
y_p = model.predict(test_pool)

In [56]:
# params = {'iterations': 100, 'depth': 2, 'loss_function': 'RMSE' }

In [63]:
# cv(train_pool, params=params)

In [76]:
y_p[y_p > 6]

array([6.0455889 , 6.0455889 , 6.85197919, 7.36446323, 6.10769395,
       7.56885034, 6.15831111, 6.09286717, 6.54790482, 6.14517401,
       6.07179226, 6.23557797, 6.19854358, 6.23557797, 6.19854358,
       6.30385653, 6.54939629, 6.6275893 ])

In [87]:
imp = model.get_feature_importance(train_pool)

In [93]:
list(zip(train_p.columns,imp))

[('Минимальная глубина CPV (2017 год)', 18.219795168909283),
 ('СуммаТендера', 4.658507899191216),
 ('Организатор', 8.560747852189357),
 ('Дата начала подачи предложений', 3.749644874382513),
 ('Тип методу закупівлі', 7.260401804088618),
 ('Тип организатора', 1.7805085153571283),
 ('Статус лота', 17.20848735057964),
 ('СтатусТендера', 4.577794717344811),
 ('РегионОрганизатора', 1.9377423894163752),
 ('Регион поставки', 2.4199014266345302),
 ('Тендер', 6.436092746595816),
 ('Дата начала периода уточнений', 4.275531349828091),
 ('Дата завершения', 8.10484826803755),
 ('Дата завершения подачи предложений', 5.888321117828066),
 ('ШагУменьшенияЛот', 4.073925381491396),
 ('СуммаГарантииПредложенияЛота', 0.8477491381256073)]

In [97]:
train_p['Статус лота'].value_counts()

Завершено лот закупівлі     286878
Неуспішний лот закупівлі    169753
Скасовано лот закупівлі      33134
Активний лот закупівлі       10272
Name: Статус лота, dtype: int64

In [81]:
make_submission(test_p, y_p, '~/results/submission_11.csv')